# Analyzing New York City subway data 

#### Author: Sushant N. More

### Data from web.mta.info/developers/turnstile.html.  Also, using a data file from weather underground obtained from Udacity website

#### Revision history: 

Sept. 15, 2017: Started writing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats
import csv

## Data cleaning

We will be analysing subway data from MTA's website for the month of May 2011 (because later I want to relate this to weather data and the weather data I have is for the month of May 2011). 

This is real-world data.  We can expect significant effort in cleaning and formatting the data. The data for a given month is in four different files.  Let's start by looking at the files to see how data is arranged. 

In [2]:
turnstile_df1 = pd.read_csv('./data/turnstile_110507_from_web_mta_info.txt')

In [3]:
turnstile_df1.head()

,A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,003143506,001087907,04-30-11.1,04:00:00,...,05-01-11,00:00:00.1,REGULAR.6,003144312,001088151,05-01-11.1,04:00:00.1,REGULAR.7,003144335,001088159
0,A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,3144353,1088177,05-01-11,12:00:00,...,05-02-11,08:00:00,REGULAR,3144941.0,1088420.0,05-02-11,12:00:00,REGULAR,3145094.0,1088753.0
1,A002,R051,02-00-00,05-02-11,16:00:00,REGULAR,3145337,1088823,05-02-11,20:00:00,...,05-03-11,16:00:00,REGULAR,3146790.0,1089417.0,05-03-11,20:00:00,REGULAR,3147615.0,1089478.0
2,A002,R051,02-00-00,05-04-11,00:00:00,REGULAR,3147798,1089515,05-04-11,04:00:00,...,05-05-11,00:00:00,REGULAR,3149281.0,1090139.0,05-05-11,04:00:00,REGULAR,3149297.0,1090145.0
3,A002,R051,02-00-00,05-05-11,08:00:00,REGULAR,3149331,1090257,05-05-11,09:04:33,...,05-05-11,12:00:00,OPEN,3149494.0,1090579.0,05-05-11,16:00:00,DOOR,3149805.0,1090652.0
4,A002,R051,02-00-00,05-05-11,20:00:00,REGULAR,3150639,1090714,05-06-11,00:00:00,...,05-06-11,20:00:00,REGULAR,3152200.0,1091283.0,NaN,NaN,NaN,NaN,NaN


The data is not conviniently arranged as expected.  And it's difficult to get sense of it by loading it into pandas data frame. Let's try looking at the file contents directly.

In [4]:
with open("./data/turnstile_110507_from_web_mta_info.txt") as myfile:
    print myfile.readlines()[0:4] 

['A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,003143506,001087907,04-30-11,04:00:00,REGULAR,003143547,001087915,04-30-11,08:00:00,REGULAR,003143563,001087935,04-30-11,12:00:00,REGULAR,003143646,001088024,04-30-11,16:00:00,REGULAR,003143865,001088083,04-30-11,20:00:00,REGULAR,003144181,001088132,05-01-11,00:00:00,REGULAR,003144312,001088151,05-01-11,04:00:00,REGULAR,003144335,001088159              \r\n', 'A002,R051,02-00-00,05-01-11,08:00:00,REGULAR,003144353,001088177,05-01-11,12:00:00,REGULAR,003144424,001088231,05-01-11,16:00:00,REGULAR,003144594,001088275,05-01-11,20:00:00,REGULAR,003144808,001088317,05-02-11,00:00:00,REGULAR,003144895,001088328,05-02-11,04:00:00,REGULAR,003144905,001088331,05-02-11,08:00:00,REGULAR,003144941,001088420,05-02-11,12:00:00,REGULAR,003145094,001088753              \r\n', 'A002,R051,02-00-00,05-02-11,16:00:00,REGULAR,003145337,001088823,05-02-11,20:00:00,REGULAR,003146168,001088888,05-03-11,00:00:00,REGULAR,003146322,001088918,05-03-11,04:00:00,REGULAR

As we can see, there are numerous data points included in each row of the MTA Subway turnstile text file. 

We want to write a function that will update each row in the text file so there is only one entry per row. So a single row from the input file will generate multiple rows. For instance the first row displayed in the above file will turn into following set of rows.

A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,003143506,001087907,
A002,R051,02-00-00,04-30-11,04:00:00,REGULAR,003143547,001087915,
A002,R051,02-00-00,04-30-11,08:00:00,REGULAR,003143563,001087935,
A002,R051,02-00-00,04-30-11,12:00:00,REGULAR,003143646,001088024,
A002,R051,02-00-00,04-30-11,16:00:00,REGULAR,003143865,001088083,
A002,R051,02-00-00,04-30-11,20:00:00,REGULAR,003144181,001088132,
A002,R051,02-00-00,05-01-11,00:00:00,REGULAR,003144312,001088151,
A002,R051,02-00-00,05-01-11,04:00:00,REGULAR,003144335,001088159

The first three elements in the input line -- A002,R051,02-00-00 -- are repeated for each of the 8 lines in the ouput file. 

Along with the above, following two preemptive operations have been done at the file below.  

1) From the time column, made a new column for Hour.  So that I can later on plot hourly data and so on

2) I wanna combine this with weather data which has the date format as yyyy-mm-dd. So I changed the date format here. 

As I found out the hard way, given the size of the data, it is really time-consuming to do in-memory computations.

In [90]:
fin1 = open("./data/turnstile_110507_from_web_mta_info.txt", 'r')
fout1 = open("./data/updated_turnstile_110507_from_web_mta_info.txt", 'w')

reader = csv.reader(fin1, delimiter = ',', quoting=csv.QUOTE_NONE)
writer = csv.writer(fout1, delimiter = ',', quoting=csv.QUOTE_NONE)

for line in reader:
    
    record1 = line[0]
    record2 = line[1]
    record3 = line[2]
    
    length = len(line)
    
    nn = (length - 1 - 7) / 5
    
    for i in range(0, nn + 1):
        
        date = line[5*i + 3]
        
        date_formatted = '20' + date.split('-')[2] + '-' + date.split('-')[0] + '-' + date.split('-')[1]
        
        lineToWrite = [record1, record2, record3, date_formatted, \
                         line[5*i + 4], int(line[5*i + 4].split(':')[0]), \
                       line[5*i + 5], line[5*i + 6], line[5*i + 7] ]
        
        writer.writerow(lineToWrite)

fin1.close()
fout1.close()

Check to see if the updated file looks as expected

In [91]:
with open("./data/updated_turnstile_110507_from_web_mta_info.txt") as myfile:
    print myfile.readlines()[0:4] 

['A002,R051,02-00-00,2011-04-30,00:00:00,0,REGULAR,003143506,001087907\r\n', 'A002,R051,02-00-00,2011-04-30,04:00:00,4,REGULAR,003143547,001087915\r\n', 'A002,R051,02-00-00,2011-04-30,08:00:00,8,REGULAR,003143563,001087935\r\n', 'A002,R051,02-00-00,2011-04-30,12:00:00,12,REGULAR,003143646,001088024\r\n']


Let's fix the data in other files all at once

In [92]:
fileloc = './data/'
filenames = [fileloc + 'turnstile_110514_from_web_mta_info.txt', \
            fileloc + 'turnstile_110521_from_web_mta_info.txt', \
            fileloc + 'turnstile_110528_from_web_mta_info.txt']

In [93]:
for i in range(0, len(filenames)):
    
    f_in = open(filenames[i], 'r')
    f_out = open(filenames[i][0:len(fileloc)] + 'updated_' + filenames[i][len(fileloc):], 'w')
    
    reader_in = csv.reader(f_in, delimiter = ',', quoting=csv.QUOTE_NONE)
    writer_out = csv.writer(f_out, delimiter = ',', quoting=csv.QUOTE_NONE)

    for line in reader_in:

        record1 = line[0]
        record2 = line[1]
        record3 = line[2]

        length = len(line)

        nn = (length - 1 - 7) / 5

        for i in range(0, nn + 1):

            date = line[5*i + 3]
        
            date_formatted = '20' + date.split('-')[2] + '-' + date.split('-')[0] + '-' + date.split('-')[1]
        
            lineToWrite = [record1, record2, record3, date_formatted, \
                             line[5*i + 4], int(line[5*i + 4].split(':')[0]), \
                           line[5*i + 5], line[5*i + 6], line[5*i + 7] ]

            writer_out.writerow(line_to_write)
            
    f_in.close()
    f_out.close()

As mentioned in the field description file, the entries in the column are as follows: 

C/A = Control Area (e.g., A002)

UNIT = Remote Unit for a station (e.g., R051)

SCP = Subunit Channel Position represents an specific address for a device (02-00-00)

DATEn = Represents the date (MM-DD-YY)

TIMEn = Represents the time (hh:mm:ss) for a scheduled audit event

DEScn = Represent the "REGULAR" scheduled audit event (occurs every 4 hours)

ENTRIESn = The comulative entry register value for a device

EXISTn = The cumulative exit register value for a device

Let's combine all the input files we generated into a single file and have a header row at the top describing the entries

In [94]:
filenamesUpdated = [fileloc + 'updated_turnstile_110507_from_web_mta_info.txt', \
            fileloc + 'updated_turnstile_110514_from_web_mta_info.txt', \
            fileloc + 'updated_turnstile_110521_from_web_mta_info.txt', \
            fileloc + 'updated_turnstile_110528_from_web_mta_info.txt']

In [95]:
with open(fileloc + 'turnstile_master_data.csv', 'w') as master_file:
    master_file.write('C/A,UNIT,SCP,DATEn,TIMEn, Hour, DESCn,ENTRIESn,EXITSn\n')
    
    for name in filenamesUpdated:
        with open(name) as infile:
            for line in infile:
                master_file.write(line)

In [96]:
with open("./data/turnstile_master_data.csv") as myfile:
    print myfile.readlines()[0:4] 

['C/A,UNIT,SCP,DATEn,TIMEn, Hour, DESCn,ENTRIESn,EXITSn\n', 'A002,R051,02-00-00,2011-04-30,00:00:00,0,REGULAR,003143506,001087907\r\n', 'A002,R051,02-00-00,2011-04-30,04:00:00,4,REGULAR,003143547,001087915\r\n', 'A002,R051,02-00-00,2011-04-30,08:00:00,8,REGULAR,003143563,001087935\r\n']


In [97]:
turnstile_df = pd.read_csv('./data/turnstile_master_data.csv')

In [98]:
turnstile_df.head()

,C/A,UNIT,SCP,DATEn,TIMEn,Hour,DESCn,ENTRIESn,EXITSn
0,A002,R051,02-00-00,2011-04-30,00:00:00,0,REGULAR,3143506,1087907
1,A002,R051,02-00-00,2011-04-30,04:00:00,4,REGULAR,3143547,1087915
2,A002,R051,02-00-00,2011-04-30,08:00:00,8,REGULAR,3143563,1087935
3,A002,R051,02-00-00,2011-04-30,12:00:00,12,REGULAR,3143646,1088024
4,A002,R051,02-00-00,2011-04-30,16:00:00,16,REGULAR,3143865,1088083


In [13]:
turnstile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854849 entries, 0 to 854848
Data columns (total 8 columns):
C/A         854849 non-null object
UNIT        854849 non-null object
SCP         854849 non-null object
DATEn       854849 non-null object
TIMEn       854849 non-null object
DESCn       854849 non-null object
ENTRIESn    854849 non-null int64
EXITSn      854849 non-null int64
dtypes: int64(2), object(6)
memory usage: 52.2+ MB


In [14]:
turnstile_df.DESCn.unique()

array(['REGULAR', 'DOOR', 'OPEN', 'TS', 'VLT', 'OPN', 'RECOVR', 'AUD',
       'LOGON', 'LGF-MAN', 'BRD', 'CHG'], dtype=object)

In [15]:
turnstile_df.DESCn.value_counts()

REGULAR    697932
DOOR        42533
OPEN        39007
RECOVR      32894
AUD         30834
TS           3927
VLT          3528
OPN          2669
LOGON        1497
BRD            12
CHG            11
LGF-MAN         5
Name: DESCn, dtype: int64

The field description key says that the 'REGULAR' in the DESCn column represents a scheduled audit event. So, let's keep only the entries which have 'REGULAR' in the DESCn column. 

In [16]:
turnstile_df_reg = turnstile_df.loc[turnstile_df['DESCn'] == 'REGULAR']

The data in the MTA Subway Turnstile data reports on the cumulative number of entries and exits per row. We would like to find the entries since the last reading (since that can be translated into subway ridership). 

We create a new column called ENTRIESn_4hourly and assign to the column the difference between ENTRIES of the currrent row and the previous row. 

In [25]:
turnstile_df_reg.loc[:,'ENTRIESn_4hourly'] = \
turnstile_df_reg.loc[:, 'ENTRIESn'] - turnstile_df_reg.loc[:, 'ENTRIESn'].shift(1) 
# using .loc is a safe way of doing this rather than using 'chained' indexing

In [18]:
turnstile_df_reg.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_4hourly
0,A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,3143506,1087907,NaN
1,A002,R051,02-00-00,04-30-11,04:00:00,REGULAR,3143547,1087915,41.0
2,A002,R051,02-00-00,04-30-11,08:00:00,REGULAR,3143563,1087935,16.0
3,A002,R051,02-00-00,04-30-11,12:00:00,REGULAR,3143646,1088024,83.0
4,A002,R051,02-00-00,04-30-11,16:00:00,REGULAR,3143865,1088083,219.0


As expected, the first variable is Nan. Let's replace it with 1. 

In [19]:
turnstile_df_reg.loc[:, 'ENTRIESn_4hourly'] = turnstile_df_reg.loc[:, 'ENTRIESn_4hourly'].fillna(1)

In [20]:
turnstile_df_reg.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_4hourly
0,A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,3143506,1087907,1.0
1,A002,R051,02-00-00,04-30-11,04:00:00,REGULAR,3143547,1087915,41.0
2,A002,R051,02-00-00,04-30-11,08:00:00,REGULAR,3143563,1087935,16.0
3,A002,R051,02-00-00,04-30-11,12:00:00,REGULAR,3143646,1088024,83.0
4,A002,R051,02-00-00,04-30-11,16:00:00,REGULAR,3143865,1088083,219.0


Let's do the same thing for the Exits and replace Nan with 0

In [21]:
turnstile_df_reg.loc[:,'EXITSn_4hourly'] = \
turnstile_df_reg.loc[:, 'EXITSn'] - turnstile_df_reg.loc[:, 'EXITSn'].shift(1) 

In [22]:
turnstile_df_reg.loc[:, 'EXITSn_4hourly'] = turnstile_df_reg.loc[:, 'EXITSn_4hourly'].fillna(0)

In [23]:
turnstile_df_reg.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_4hourly,EXITSn_4hourly
0,A002,R051,02-00-00,04-30-11,00:00:00,REGULAR,3143506,1087907,1.0,0.0
1,A002,R051,02-00-00,04-30-11,04:00:00,REGULAR,3143547,1087915,41.0,8.0
2,A002,R051,02-00-00,04-30-11,08:00:00,REGULAR,3143563,1087935,16.0,20.0
3,A002,R051,02-00-00,04-30-11,12:00:00,REGULAR,3143646,1088024,83.0,89.0
4,A002,R051,02-00-00,04-30-11,16:00:00,REGULAR,3143865,1088083,219.0,59.0


For future analysis, it's convenient if we convert TIMEn into the hour of the day. 